<a href="https://colab.research.google.com/github/KenzioDG/Colab-Projects/blob/main/Persona_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pada kasus ini, diberikan suatu dataset yang berisi abstrak dari berbagai sumber studi. Tujuan kasus ini adalah untuk membuat suatu metode pengelompokkan dokumen ini ke kategori masing-masing dengan memberikan coherence score dan persona analysis, karena pada dataset ini tidak terdapat label sama sekali

Untuk mengelompokkan topik dari abstrak-abstrak yang diberikan tanpa menggunakan label, kita dapat menggunakan metode pengelompokan topik unsupervised, dalam kasus ini digunakan **Latent Dirichlet Allocation (LDA).** LDA adalah model generatif probabilistik yang digunakan untuk menganalisis topik dalam koleksi dokumen. Model ini merupakan salah satu metode yang populer dalam topic modeling yang bertujuan untuk mengidentifikasi topik-topik yang ada dalam dokumen secara otomatis.

In [ ]:
!gdown 13qGCLFSbXb8v231vsjjZh3GwlIyTAvhH

Downloading...
From: https://drive.google.com/uc?id=13qGCLFSbXb8v231vsjjZh3GwlIyTAvhH
To: /content/data_3A.csv
100% 96.2k/96.2k [00:00<00:00, 105MB/s]



- `stopwords`: Pada kasus ini daftar stopwords akan digunakan dalam tahap preprocessing untuk menghapus stopwords dari teks abstrak.

- `wordnet`: Dalam kasus ini, WordNet digunakan untuk melakukan lemmatisasi, yaitu mengubah kata-kata ke bentuk dasar mereka, seperti mengubah "running" menjadi "run".

- `punkt`: Dalam kasus ini, pemodelan tokenisasi diperlukan untuk memisahkan teks abstrak menjadi kata-kata sebelum dilakukan tahap preprocessing.

In [ ]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
from gensim.models import CoherenceModel
from nltk.tokenize import word_tokenize


# Download stopwords and lemmatizer data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Load data
data = pd.read_csv('data_3A.csv')

## Preprocessing

Stopwords dihapus dan kata-kata dibiarkan dalam bentuk dasar mereka dengan melakukan lemmatisasi menggunakan lemmatizer. Kata-kata yang ada dalam stop_words akan dihapus dari hasil lemmatisasi.

Setelah proses preprocessing, dilakukan pembuatan kamus (dictionary) dan korpus.
Kamus (dictionary) dibuat dengan menggunakan `corpora.Dictionary()` pada kolom `processed_abstract` dari dataframe data. Ini akan membuat mapping antara kata-kata unik dalam teks ke indeks numerik.

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Cleaning
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs or links
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove non-alphabetic characters (keeping whitespace)
    text = re.sub(r"\s+", " ", text)  # Remove extra whitespace
    text = text.lower()  # Convert to lowercase

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]

    # Join tokens back into string
    processed_text = ' '.join(tokens)

    return processed_text

data['processed_abstract'] = data['ABSTRACT'].apply(preprocess_text)

# Create dictionary and corpus
texts = [text.split() for text in data['processed_abstract']]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          100 non-null    int64 
 1   ABSTRACT            100 non-null    object
 2   processed_abstract  100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


Hasil text preprocessing kira-kira seperti di bawah ini

In [ ]:
data['processed_abstract']

0     predictive model allow subjectspecific inferen...
1     rotation invariance translation invariance gre...
2     introduce develop notion spherical polyharmoni...
3     stochastic landaulifshitzgilbert llg equation ...
4     fouriertransform infrared ftir spectrum sample...
                            ...                        
95    paper presented novel convolutional neural net...
96    variety representation learning approach inves...
97    motivated perelmans pseudo locality theorem ri...
98    bound exponential sum appears study irregulari...
99    investigate effect dimensional crossover groun...
Name: processed_abstract, Length: 100, dtype: object

## LDA Model
Pada kode di bawah, dilakukan iterasi untuk melatih model LDA dengan jumlah topik yang berbeda dan juga analisis terkait hasilnya. Berikut adalah penjelasan langkah-langkah yang dilakukan dalam kode tersebut:

- `num_topics_list` didefinisikan sebagai daftar yang berisi jumlah topik yang akan diuji, dalam contoh kode ini adalah [2, 3, 5, 7, 8]. Maka berarti model LDA akan dilatih dengan mengelompokkan ke topik sebanyak 2, 3, 5, 7, dan 8 buah secara terpisah

- Dalam setiap iterasi, model LDA (lda_model) dilatih dengan menggunakan fungsi models.LdaModel(). Parameternya adalah korpus BoW yang telah dibuat sebelumnya, num_topics, id2word (kamus yang digunakan), dan passes (jumlah iterasi pada data).

- Setelah melatih model LDA, coherence score dihitung menggunakan CoherenceModel dengan menggunakan model LDA, teks abstrak yang telah diproses

- Setelah mendapatkan model LDA, persona analysis dilakukan dengan mencetak topik-topik yang dihasilkan oleh model menggunakan print_topics.

### Berikut ini adalah pengelompokan dan output Coherence Score

In [ ]:
# Define number of topics
num_topics_list = [2, 4, 6, 8, 10]

for num_topics in num_topics_list:
    print(f"Number of Topics: {num_topics}")

    # Train LDA model
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=50, random_state=69)

    # Evaluate coherence score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    print(f"Coherence Score: {coherence_score}\n")

    # Perform persona analysis
    topics = lda_model.print_topics(num_topics=num_topics)
    for topic in topics:
        topic_num = topic[0] + 1  # Add 1 to start topic number from 1
        print(f"Topic {topic_num}: {topic[1]}\n")

    # Count data in each topic
    topic_counts = [0] * num_topics
    total_docs = 0

    for doc in corpus:
        topic_distribution = lda_model[doc]
        sorted_topics = sorted(topic_distribution, key=lambda x: x[1], reverse=True)
        top_topic = sorted_topics[0][0]
        topic_counts[top_topic] += 1
        total_docs += 1

    # Print percentage of data in each topic
    for i, count in enumerate(topic_counts):
        percentage = (count / total_docs) * 100
        print(f"Topic {i+1}: {percentage:.2f}%")

    print()


Number of Topics: 2
Coherence Score: 0.290093230451281

Topic 1: 0.006*"model" + 0.005*"system" + 0.004*"method" + 0.004*"approach" + 0.004*"result" + 0.003*"used" + 0.003*"application" + 0.003*"performance" + 0.003*"sample" + 0.003*"data"

Topic 2: 0.009*"model" + 0.006*"method" + 0.005*"state" + 0.004*"network" + 0.004*"result" + 0.004*"function" + 0.004*"also" + 0.003*"show" + 0.003*"based" + 0.003*"equation"

Topic 1: 43.00%
Topic 2: 57.00%

Number of Topics: 4
Coherence Score: 0.34938396677322825

Topic 1: 0.007*"system" + 0.007*"model" + 0.006*"approach" + 0.005*"framework" + 0.005*"used" + 0.004*"show" + 0.004*"result" + 0.003*"well" + 0.003*"two" + 0.003*"study"

Topic 2: 0.009*"model" + 0.006*"function" + 0.006*"network" + 0.005*"formation" + 0.005*"using" + 0.004*"x" + 0.004*"state" + 0.004*"domain" + 0.004*"condition" + 0.003*"detection"

Topic 3: 0.010*"method" + 0.007*"model" + 0.006*"state" + 0.005*"result" + 0.004*"group" + 0.004*"data" + 0.004*"q" + 0.004*"show" + 0.004

Pada kode di atas, LDA mengaelompokkan data ke 3 label dan 5 label. Untuk menganalisa hasil maka dicetaklah kata-kata dengan bobot yang paling dominan di sini. Contohnya pada saat LDA mengelompokkan kepada 3 label, topik 1 memiliki bobot:

`Topic 1: 0.007*"model" + 0.006*"state" + 0.006*"system" + 0.005*"based" + 0.005*"method" + 0.004*"used" + 0.004*"result" + 0.004*"approach" + 0.004*"show" + 0.004*"term"
`</br>

Dalam representasi topik tersebut, setiap kata memiliki bobot yang menunjukkan seberapa penting kata tersebut dalam konteks topik tertentu. Bobot ini dihitung berdasarkan frekuensi kemunculan kata tersebut dalam korpus dan berapa banyak kata tersebut terkait dengan topik tersebut. model, state, system, based, dll, merupakan kata dengan bobot yang menunjukan bahwa kata-kata ini sangat relevan dengan topik 1.

Coherence score didapatkan dengan menggunakan metode c_v. Untuk ukurannya, biasanya menggunakan skala berikut:
- Skor di bawah 0,2: Koherensi yang rendah atau tidak relevan.
- Skor antara 0,2 hingga 0,4: Koherensi yang cukup baik.
- Skor antara 0,4 hingga 0,6: Koherensi yang baik.
- Skor di atas 0,6: Koherensi yang sangat baik.

Namun kita dapatkan skor berikut:
- 2 topik = Coherence Score: 0.290093230451281
- 4 topik = Coherence Score: 0.34938396677322825
- 6 topik = Coherence Score: 0.4054504966124131
- 8 topik = Coherence Score: 0.4038018046834254
- 10 topik = Coherence Score: 0.3892700625310904

Berarti hasil yang lebih optimal adalah dengan 6 jumlah topik.

### Persona Analysis

 Persona analysis dapat dilakukan dengan mengidentifikasi dan menganalisis topik yang paling relevan atau signifikan untuk setiap kelompok pengguna atau audiens. Dalam hal ini, topik-topik yang muncul dalam analisis topik dapat digunakan sebagai fitur untuk menggambarkan karakteristik dan preferensi kelompok pengguna atau audiens yang berbeda.

In [ ]:
# Train LDA model
num_topics = int(input("Enter the total number of topics: "))
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=50, random_state = 69)

# Print topics
topics = lda_model.print_topics(num_topics=num_topics)
for topic in topics:
    topic_num = topic[0] + 1  # Add 1 to start topic number from 1
    print(f"Topic {topic_num}: {topic[1]}\n")

# Prompt user to analyze a specific topic
topic_to_analyze = int(input("Enter the topic number to analyze: "))

# Print abstracts in the selected topic
topic_documents = []
for i, doc in enumerate(corpus):
    topic_distribution = lda_model[doc]
    sorted_topics = sorted(topic_distribution, key=lambda x: x[1], reverse=True)
    top_topic = sorted_topics[0][0] + 1  # Add 1 to start topic number from 1
    if top_topic == topic_to_analyze:
        topic_documents.append(data['ABSTRACT'][i])

print(f"\nDocuments in Topic {topic_to_analyze}:")
for document in topic_documents:
    print("- " + document)
    print()


Enter the total number of topics: 6
Topic 1: 0.006*"approach" + 0.006*"model" + 0.005*"epidemic" + 0.004*"system" + 0.004*"show" + 0.004*"robot" + 0.004*"paper" + 0.004*"human" + 0.004*"algorithm" + 0.004*"performance"

Topic 2: 0.009*"network" + 0.007*"model" + 0.006*"system" + 0.005*"function" + 0.005*"complex" + 0.005*"using" + 0.004*"flow" + 0.004*"detection" + 0.004*"data" + 0.004*"region"

Topic 3: 0.011*"method" + 0.007*"result" + 0.006*"state" + 0.006*"model" + 0.005*"group" + 0.005*"sample" + 0.005*"q" + 0.004*"show" + 0.004*"data" + 0.004*"drone"

Topic 4: 0.009*"model" + 0.007*"equation" + 0.007*"sample" + 0.006*"system" + 0.006*"based" + 0.006*"adversarial" + 0.005*"measure" + 0.005*"proposed" + 0.005*"network" + 0.005*"posterior"

Topic 5: 0.007*"method" + 0.006*"model" + 0.006*"learning" + 0.006*"representation" + 0.005*"state" + 0.005*"also" + 0.005*"formation" + 0.005*"star" + 0.005*"application" + 0.004*"approach"

Topic 6: 0.012*"model" + 0.008*"sequence" + 0.005*"cel

Output di atas didapatkan dengan cara memilih topik 3 dari total 6 total topik. Setelah proses pemeriksaan dapat diperkirakan bahwa topik 3 ini mencakup topik pada ranah matematika. Namun patut diakui bahwa pengelompokan persona analysis cukup sulit dilakukan tanpa pengetahuan mendalam dari ranah paper tersebut.
Maka dengan melakukan persona analysis untuk abstrak yang terkelompokkan pada topik 2, kelompok ini dapat teridentifikasi secara manual dan ditujukan kepada pihak atau lembaga yang mendalami matematika. Bahkan untuk tahap selanjutnya hasil ini dapat digunakan untuk proses labelling, sehingga data ini dapat dipakai untuk membuat model prediksi supervised.

Namun, menurut saya pribadi hasilnya kurang valid, ada banyak dokumen yang menurut saya seharusnya tidak termasuk dalam topik sama, seperti biologi dengan deep learning. Mungkin perlu dilakukan analisis lebih lanjut lagi untuk hasil elbih bagus. Saya berasumsi kalau hal hal ini terjadi karena kekurangan data yang hanya berjumlah 100 row. Untuk pengelompokkan lebih baik patut dilaksanakan percobaan baru dengan data yang jauh lebih banyak.